# Indexing Method Comparison (LlamaIndex)

This notebook aggregates method outputs from `results/` and provides ranking + category summaries.


In [ ]:
%pip install -q pandas


In [ ]:
from pathlib import Path
import json
import pandas as pd

repo_root = Path.cwd()
if not (repo_root / "results").exists():
    repo_root = repo_root.parent

results_dir = repo_root / "results"


In [ ]:
EXPECTED_METHOD_IDS = ['1.1_fixed_length_chunking', '1.2_sliding_window_overlap', '1.3_adaptive_semantic_chunking', '1.4_query_aware_dynamic_chunking', '1.5_small_to_big_hierarchical_chunking', '1.6_sentence_level_indexing', '2.1_flat_dense_index', '2.2_approximate_nearest_neighbor_index', '2.3_quantized_vector_index', '2.4_hybrid_dense_partition_index', '3.1_inverted_index', '3.2_sparse_dense_fusion_index', '3.3_weighted_term_index', '4.1_tree_structured_index', '4.2_summary_augmented_index', '4.3_structural_aware_index', '5.1_knowledge_graph_index', '5.2_entity_centric_index', '5.3_semantic_similarity_graph', '5.4_multi_hop_retrieval_graph', '6.1_topic_partition_index', '6.2_semantic_hashing_index', '7.1_time_decayed_index', '7.2_sliding_temporal_window_index', '7.3_real_time_streaming_index', '8.1_trust_weighted_index', '8.2_outlier_aware_vector_index', '8.3_drift_aware_index', '9.1_distributional_embedding_index', '9.2_confidence_propagating_index', '10.1_joint_embedding_index', '10.2_cross_modal_linked_index', '10.3_scene_aware_media_index', '11.1_self_tuning_index', '11.2_feedback_driven_index', '11.3_continual_re_embedding_index', '12.1_causal_graph_index', '12.2_explanation_aware_index', '12.3_plan_oriented_index']

rows = []
for path in sorted(results_dir.glob("*.json")):
    payload = json.loads(path.read_text(encoding="utf-8"))
    metrics = payload.get("metrics", {})
    rows.append(
        {
            "method_id": payload.get("method_id"),
            "method_name": payload.get("method_name"),
            "category": payload.get("category"),
            "embedding_source": payload.get("embedding_source"),
            "precision_at_k": metrics.get("precision_at_k", 0.0),
            "recall_at_k": metrics.get("recall_at_k", 0.0),
            "mrr": metrics.get("mrr", 0.0),
            "ndcg_at_k": metrics.get("ndcg_at_k", 0.0),
            "nodes": payload.get("nodes", 0),
            "result_file": path.name,
        }
    )

score_df = pd.DataFrame(rows)
score_df.sort_values(["mrr", "ndcg_at_k"], ascending=False).reset_index(drop=True)


In [ ]:
if len(score_df) == 0:
    print("No results found in results/. Run method notebooks first.")
else:
    category_summary = (
        score_df.groupby("category", as_index=False)[["precision_at_k", "recall_at_k", "mrr", "ndcg_at_k"]]
        .mean()
        .sort_values("mrr", ascending=False)
    )
    display(category_summary)


In [ ]:
available = set(score_df["method_id"].tolist()) if len(score_df) else set()
missing = sorted(set(EXPECTED_METHOD_IDS) - available)

print(f"Expected methods: {len(EXPECTED_METHOD_IDS)}")
print(f"Available result files: {len(available)}")
print("Missing methods:")
missing
